## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Strat Convolutional:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

Strat de Pooling:
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
* torch.nn.AveragePool2d(kernel_size, stride=None, padding=0)

Strat de Adaptive Pool, intalnit adesea si ca Global Pool:
* torch.nn.AdaptiveAvgPool2d(output_size)
* torch.nn.AdaptiveMaxPool2d(output_size)

Strat de liniarizare:

* torch.nn.Flatten()



In [1]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizat pentru a reduce dimensiunea input-ului la una prestabilita, util cand marimea input-ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # hint: folositi stride si padding
*   [1, 10, 32, 32]
*   [1, 3, 2, 2]



In [2]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

### Am utilizat formula (n + 2*p - f)/stride + 1 = output
### Completati codul pentru cerinta aici
layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(4,4))
print("Conv result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, padding=1, kernel_size=(7,7), stride=(3,3))
print("Conv result shape",layer(dummy_input_tensor).shape)

layer = nn.AdaptiveAvgPool2d(output_size=(2,2))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

Conv result shape torch.Size([1, 10, 25, 25])
Conv result shape torch.Size([1, 10, 32, 32])
Global Pool result shape torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [3]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
CUDA is available!  Training on GPU ...


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def collate_fn(examples):
  ### Completati codul pentru cerinta aici
  """
    Functia primeste un batch de exemple pe care trebuie sa le transforme in tensori
      si sa le puna intr-un batch de tip torch.Tensor.
  """
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    # In linia de mai jos imaginea este normalizata astfel incat sa aiba toate valorile in 
    # [-1, 1] in loc de [0, 255]
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    normalized_tensor_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(normalized_tensor_image)
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  #print(len(processed_images))
  #processed_images = torch.FloatTensor(processed_images)
  #print(processed_images[0].shape)
  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)
  return torch_images, torch_labels


train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)


dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images[0].shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([3, 32, 32])
torch.Size([500])


## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii
    * Liniarizati iesirea celui de-al doilea strat convolutional
    * Adaugat stratul final de tipul 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [5]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    ### Completati codul pentru cerinta aici
    # convolutional layer (sees 32x32x3 image tensor)
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    #self.batchnorm1 = nn.BatchNorm2d(16)
    # convolutional layer (sees 16x16x16 tensor)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

    # max pooling layer
    self.pool = nn.MaxPool2d(2, 2)
    self.linearize = nn.Flatten()
    # linear layer (64 * 12 * 3 -> 1000)
    self.fc1 = nn.Linear(8192, 10)
    self.activation = nn.ReLU()


  def forward(self,x):
    ### Completati codul pentru cerinta aici
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.pool(x)
    x = self.linearize(x)
    x = self.activation(x)
    x = self.fc1(x)
    x = self.activation(x)
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Initializati numarul de epoci
  * Initializati retea
  * Initializati optimizator
  * Initializati functia de cost

In [6]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
network = Net()

if train_on_gpu:
    network.cuda()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum = 0.9)

"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie 
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [7]:
def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    if train_on_gpu:
            test_images, test_labels = test_images.cuda(), test_labels.cuda()
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [ ]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.016284227371216
Acuratetea la finalul epocii 1 este 30.48%
Loss-ul la finalul epocii 1 are valoarea 1.8695898056030273
Acuratetea la finalul epocii 2 este 37.05%
Loss-ul la finalul epocii 2 are valoarea 1.8183363676071167
Acuratetea la finalul epocii 3 este 39.81%
Loss-ul la finalul epocii 3 are valoarea 1.6757789850234985
Acuratetea la finalul epocii 4 este 42.03%
Loss-ul la finalul epocii 4 are valoarea 1.6433016061782837
Acuratetea la finalul epocii 5 este 43.46%
Loss-ul la finalul epocii 5 are valoarea 1.5122630596160889
Acuratetea la finalul epocii 6 este 45.83%
Loss-ul la finalul epocii 6 are valoarea 1.5558327436447144
Acuratetea la finalul epocii 7 este 47.04%
Loss-ul la finalul epocii 7 are valoarea 1.480910301208496
Acuratetea la finalul epocii 8 este 49.27%
Loss-ul la finalul epocii 8 are valoarea 1.4283952713012695
Acuratetea la finalul epocii 9 este 50.15%
Loss-ul la finalul epocii 9 are valoarea 1.3720030784606934
Acuratetea la f

## Reteaua LeNet

### Cerinte
  * **(2.5p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet


In [25]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    ### Completati codul pentru cerinta aici
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=(1,1))
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1))
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1))
    self.batchnorm1 = nn.BatchNorm2d(6)
    self.batchnorm2 = nn.BatchNorm2d(16)
    self.batchnorm3 = nn.BatchNorm2d(120)
    self.avgpool1 = nn.AvgPool2d(2,2)
    self.activation1 = nn.ReLU()
    self.activation2 = nn.LeakyReLU()
    self.linear1 = nn.Linear(120,84)
    self.linear2 = nn.Linear(84,10)
    self.liniarize = nn.Flatten()

  def forward(self,x):
    ### Completati codul pentru cerinta aici
    x = self.avgpool1(self.batchnorm1(self.activation1(self.conv1(x))))
    x = self.avgpool1(self.batchnorm2(self.activation1(self.conv2(x))))
    x = self.batchnorm3(self.activation1(self.conv3(x)))
    x = self.liniarize(x)
    x = self.activation1(self.linear1(x))
    x = self.activation2(self.linear2(x))
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * **(0.5p)** Redefiniti obiectele necesare pentru a antrena reteaua LeNet

In [26]:
import torch.optim as optim
import torch
    
# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

if train_on_gpu:
    lenet.cuda()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=0.001, momentum=0.9)

# Completati aici codul pentru a face gradientii zero
lenet_optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [27]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

""" tanh Relu 0.001
Loss-ul la finalul epocii 0 are valoarea 2.0796093940734863
Acuratetea la finalul epocii 1 este 29.20%
Loss-ul la finalul epocii 1 are valoarea 1.9524906873703003
Acuratetea la finalul epocii 2 este 32.91%
Loss-ul la finalul epocii 2 are valoarea 1.900344967842102
Acuratetea la finalul epocii 3 este 35.40%
Loss-ul la finalul epocii 3 are valoarea 1.8716014623641968
Acuratetea la finalul epocii 4 este 36.63%
Loss-ul la finalul epocii 4 are valoarea 1.7920197248458862
Acuratetea la finalul epocii 5 este 37.74%
Loss-ul la finalul epocii 5 are valoarea 1.7456880807876587
Acuratetea la finalul epocii 6 este 39.06%
Loss-ul la finalul epocii 6 are valoarea 1.749069333076477
Acuratetea la finalul epocii 7 este 39.87%
Loss-ul la finalul epocii 7 are valoarea 1.7060997486114502
Acuratetea la finalul epocii 8 este 40.56%
Loss-ul la finalul epocii 8 are valoarea 1.6933534145355225
Acuratetea la finalul epocii 9 este 40.82%
Loss-ul la finalul epocii 9 are valoarea 1.8033382892608643
Acuratetea la finalul epocii 10 este 41.33%
Loss-ul la finalul epocii 10 are valoarea 1.7434660196304321
Acuratetea la finalul epocii 11 este 41.75%
Loss-ul la finalul epocii 11 are valoarea 1.605223298072815
Acuratetea la finalul epocii 12 este 42.38%
Loss-ul la finalul epocii 12 are valoarea 1.6265277862548828
Acuratetea la finalul epocii 13 este 42.76%
Loss-ul la finalul epocii 13 are valoarea 1.6651031970977783
Acuratetea la finalul epocii 14 este 42.74%
Loss-ul la finalul epocii 14 are valoarea 1.5793588161468506
Acuratetea la finalul epocii 15 este 43.66%
Loss-ul la finalul epocii 15 are valoarea 1.550754189491272
Acuratetea la finalul epocii 16 este 43.92%
Loss-ul la finalul epocii 16 are valoarea 1.658993124961853
Acuratetea la finalul epocii 17 este 44.09%
Loss-ul la finalul epocii 17 are valoarea 1.5809075832366943
Acuratetea la finalul epocii 18 este 44.48%
Loss-ul la finalul epocii 18 are valoarea 1.5090138912200928
Acuratetea la finalul epocii 19 este 44.98%
Loss-ul la finalul epocii 19 are valoarea 1.5281784534454346
Acuratetea la finalul epocii 20 este 45.32%

""" 
""" ReLU ReLU
Loss-ul la finalul epocii 0 are valoarea 2.208341598510742
Acuratetea la finalul epocii 1 este 24.70%
Loss-ul la finalul epocii 1 are valoarea 2.031294584274292
Acuratetea la finalul epocii 2 este 32.35%
Loss-ul la finalul epocii 2 are valoarea 1.830610990524292
Acuratetea la finalul epocii 3 este 36.40%
Loss-ul la finalul epocii 3 are valoarea 1.7299917936325073
Acuratetea la finalul epocii 4 este 38.80%
Loss-ul la finalul epocii 4 are valoarea 1.6747359037399292
Acuratetea la finalul epocii 5 este 41.24%
Loss-ul la finalul epocii 5 are valoarea 1.6446526050567627
Acuratetea la finalul epocii 6 este 43.16%
Loss-ul la finalul epocii 6 are valoarea 1.525226354598999
Acuratetea la finalul epocii 7 este 44.09%
Loss-ul la finalul epocii 7 are valoarea 1.4689027070999146
Acuratetea la finalul epocii 8 este 45.29%
Loss-ul la finalul epocii 8 are valoarea 1.4290144443511963
Acuratetea la finalul epocii 9 este 46.58%
Loss-ul la finalul epocii 9 are valoarea 1.452158808708191
Acuratetea la finalul epocii 10 este 48.08%
"""

""" ReLU LeakyReLU
Loss-ul la finalul epocii 0 are valoarea 2.2462141513824463
Acuratetea la finalul epocii 1 este 19.70%
Loss-ul la finalul epocii 1 are valoarea 2.124427080154419
Acuratetea la finalul epocii 2 este 27.36%
Loss-ul la finalul epocii 2 are valoarea 2.0321152210235596
Acuratetea la finalul epocii 3 este 30.28%
Loss-ul la finalul epocii 3 are valoarea 1.9179110527038574
Acuratetea la finalul epocii 4 este 33.05%
Loss-ul la finalul epocii 4 are valoarea 1.788777232170105
Acuratetea la finalul epocii 5 este 36.04%
Loss-ul la finalul epocii 5 are valoarea 1.6475142240524292
Acuratetea la finalul epocii 6 este 38.13%
Loss-ul la finalul epocii 6 are valoarea 1.6639798879623413
Acuratetea la finalul epocii 7 este 41.10%
Loss-ul la finalul epocii 7 are valoarea 1.5541795492172241
Acuratetea la finalul epocii 8 este 43.65%
Loss-ul la finalul epocii 8 are valoarea 1.5308094024658203
Acuratetea la finalul epocii 9 este 44.95%
Loss-ul la finalul epocii 9 are valoarea 1.470102310180664
Acuratetea la finalul epocii 10 este 46.52%
"""


Loss-ul la finalul epocii 0 are valoarea 2.2462141513824463
Acuratetea la finalul epocii 1 este 19.70%
Loss-ul la finalul epocii 1 are valoarea 2.124427080154419
Acuratetea la finalul epocii 2 este 27.36%
Loss-ul la finalul epocii 2 are valoarea 2.0321152210235596
Acuratetea la finalul epocii 3 este 30.28%
Loss-ul la finalul epocii 3 are valoarea 1.9179110527038574
Acuratetea la finalul epocii 4 este 33.05%
Loss-ul la finalul epocii 4 are valoarea 1.788777232170105
Acuratetea la finalul epocii 5 este 36.04%
Loss-ul la finalul epocii 5 are valoarea 1.6475142240524292
Acuratetea la finalul epocii 6 este 38.13%
Loss-ul la finalul epocii 6 are valoarea 1.6639798879623413
Acuratetea la finalul epocii 7 este 41.10%
Loss-ul la finalul epocii 7 are valoarea 1.5541795492172241
Acuratetea la finalul epocii 8 este 43.65%
Loss-ul la finalul epocii 8 are valoarea 1.5308094024658203
Acuratetea la finalul epocii 9 este 44.95%
Loss-ul la finalul epocii 9 are valoarea 1.470102310180664
Acuratetea la fi

' ReLU LeakyReLU\n'

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:


* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu augmentarile mentionate mai sus in cadrul ultimei retele definite pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
